In [2]:
import pandas as pd
import numpy as np

In [3]:
ls

1.0-ildar-initial-data-exploration.ipynb


In [5]:
df = pd.read_csv('../data/raw/train_df.csv')

In [6]:
pd.options.display.max_columns = 100

In [7]:
df.head()

,crop_path,warp_path,person_id,video_id,track_id,frame_id,bbox_x,bbox_y,bbox_w,bbox_h,x0,y0,x1,y1,x2,y2,x3,y3,x4,y4,is_val
0,00050593d7333113d45bb44b39379621/DJGvVAbyBBc/2...,00050593d7333113d45bb44b39379621/DJGvVAbyBBc/2...,00050593d7333113d45bb44b39379621,DJGvVAbyBBc,250f8b8ee05b39460dfd8f99457c4dc7,0,141,195,141,195,60,75,126,81,108,108,63,141,123,144,True
1,00050593d7333113d45bb44b39379621/DJGvVAbyBBc/2...,00050593d7333113d45bb44b39379621/DJGvVAbyBBc/2...,00050593d7333113d45bb44b39379621,DJGvVAbyBBc,250f8b8ee05b39460dfd8f99457c4dc7,1,147,198,147,198,63,78,129,81,114,111,66,144,129,147,True
2,00050593d7333113d45bb44b39379621/DJGvVAbyBBc/2...,00050593d7333113d45bb44b39379621/DJGvVAbyBBc/2...,00050593d7333113d45bb44b39379621,DJGvVAbyBBc,250f8b8ee05b39460dfd8f99457c4dc7,2,144,192,144,192,57,75,126,78,108,105,60,138,126,141,True
3,00050593d7333113d45bb44b39379621/DJGvVAbyBBc/2...,00050593d7333113d45bb44b39379621/DJGvVAbyBBc/2...,00050593d7333113d45bb44b39379621,DJGvVAbyBBc,250f8b8ee05b39460dfd8f99457c4dc7,3,165,207,165,207,39,90,114,75,78,117,51,153,129,141,True
4,00050593d7333113d45bb44b39379621/DJGvVAbyBBc/2...,00050593d7333113d45bb44b39379621/DJGvVAbyBBc/2...,00050593d7333113d45bb44b39379621,DJGvVAbyBBc,250f8b8ee05b39460dfd8f99457c4dc7,4,162,204,162,204,33,84,111,72,72,114,45,150,123,138,True


In [8]:
descr = np.load('../data/raw/train_df_descriptors.npy')

In [9]:
descr.shape

(154420, 512)

In [10]:
df.shape

(154420, 21)

In [11]:
track_order = pd.read_csv('../data/raw/train_df_track_order_df.csv')

In [13]:
track_order.head()

,track_id,person_id
0,250f8b8ee05b39460dfd8f99457c4dc7,00050593d7333113d45bb44b39379621
1,f53c6de832c48eb35610f84d60409ed5,00050593d7333113d45bb44b39379621
2,93be3adedcbc2a1e20559d1116fdbb8c,00050593d7333113d45bb44b39379621
3,83d0df1195c88ba61bb14e1e4422ddd7,00050593d7333113d45bb44b39379621
4,a01a124be27134c07d10d9bc8e97cd09,00050593d7333113d45bb44b39379621


In [14]:
df.is_val.value_counts()

False    115140
True      39280
Name: is_val, dtype: int64

In [15]:
len(np.unique(df.person_id.values))

935

In [16]:
len(np.unique(df.track_id.values))

15442

In [17]:
len(track_order.track_id.values)

15442

In [18]:
len(np.unique(track_order.track_id.values))

15442

In [21]:
# averaged train descriptors from the example
avg_train_desc = np.load('../models/baseline/train_df_agg_descriptors.npy')

In [22]:
avg_train_desc.shape

(15442, 512)

### From where do we get pairs???
Мы считаем по самому тупому методу - для каждого чувака берем ембединги по каждому его треку и сравниваем с каждой фоткой из его референс имеджесов и считаем что это класс 1.
Потом также сравниваем эмбеддинги по каждому треку с со всеми не его фотками и считаем что это класс 0
Далее строим roc-curve по этим дистанциям и меткам классов и смотрим значение в fpr = 10^(-6)

In [32]:
for p_id, subdf in track_order.groupby('person_id'):
    print(len(subdf))
    break

20


In [27]:
train_gt_df = pd.read_csv('../data/raw/train_gt_df.csv')

In [29]:
train_gt_df.head()

,crop_path,warp_path,person_id,person_img_id,bbox_x,bbox_y,bbox_w,bbox_h,x0,y0,x1,y1,x2,y2,x3,y3,x4,y4
0,cb0b5cb9792a436f8cc6f8d2efc26948/gt_imgs/0000_...,cb0b5cb9792a436f8cc6f8d2efc26948/gt_imgs/0000_...,cb0b5cb9792a436f8cc6f8d2efc26948,0,167,193,167,193,32,94,97,73,62,110,49,155,114,136
1,cb0b5cb9792a436f8cc6f8d2efc26948/gt_imgs/0001_...,cb0b5cb9792a436f8cc6f8d2efc26948/gt_imgs/0001_...,cb0b5cb9792a436f8cc6f8d2efc26948,1,285,383,285,383,113,157,227,141,207,184,128,282,248,269
2,cb0b5cb9792a436f8cc6f8d2efc26948/gt_imgs/0002_...,cb0b5cb9792a436f8cc6f8d2efc26948/gt_imgs/0002_...,cb0b5cb9792a436f8cc6f8d2efc26948,2,183,235,183,235,48,110,128,107,90,137,56,186,129,182
3,cb0b5cb9792a436f8cc6f8d2efc26948/gt_imgs/0003_...,cb0b5cb9792a436f8cc6f8d2efc26948/gt_imgs/0003_...,cb0b5cb9792a436f8cc6f8d2efc26948,3,231,281,231,281,49,121,152,119,94,152,48,214,157,213
4,cb0b5cb9792a436f8cc6f8d2efc26948/gt_imgs/0004_...,cb0b5cb9792a436f8cc6f8d2efc26948/gt_imgs/0004_...,cb0b5cb9792a436f8cc6f8d2efc26948,4,220,285,220,285,53,131,152,119,106,169,57,214,166,203


In [30]:
len(np.unique(track_order.person_id.values))

935

In [ ]:
# algorithm

agg_descr_arr = np.load(conf.predicted_descr_path)
test_track_order_df = pd.read_csv(conf.test_track_order_df)
test_gt_df = pd.read_csv(conf.test_gt_df_path)
gt_descr = np.load(conf.gt_descriptors_path)

dist_arr = []
labels_arr = []

#for each person in test_track_order_file
for curr_person_id, subdf in tqdm(test_track_order_df.groupby('person_id')):
    # get gt_df that person
    gt_subdf = test_gt_df[test_gt_df.person_id == curr_person_id]
    # for each track with THIS person 
    for idx in subdf.index.values:
        # for each gt image of that person
        for jdx in gt_subdf.index.values:
            # compute the norm of the diff vector on some aggregated descriptor of the track
            # by that trach id with the gt_descriptor of a single photo
            dist = np.linalg.norm(agg_descr_arr[idx] - gt_descr[jdx])
            # add this to distances
            dist_arr.append(dist)
            # add positive label
            labels_arr.append(1)
    
    # get gt_df of not this person (all others)
    gt_subdf = test_gt_df[test_gt_df.person_id != curr_person_id]
    # for each track with this person 
    for idx in subdf.index.values:
        # for each gt image of ANY other person
        for jdx in gt_subdf.index.values:
            # compute the norm of the diff vector on some aggregated descriptor (from the NNet)of the track
            # by that track id with the gt_descriptor of a single photo
            dist = np.linalg.norm(agg_descr_arr[idx] - gt_descr[jdx])
            # add this to distances
            dist_arr.append(dist)
            # add positive label
            labels_arr.append(0)